# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [ ]:
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики доля правильных ответов (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
class Node:
    def __init__(self, chances=None):
        self.chances = chances
        self.depth = 1
        self.bad = False
        self.j = None
        self.t = None
        self.right = None
        self.left = None

    def _grow_node(self, chances):
        self.depth += 1
        return Node(chances)


class Tree:
    def __init__(self, max_tree_depth = 5, max_tree_leafs = 5, min_leaf_obj = 5):
        self.max_tree_depth = max_tree_depth
        self.min_leaf_obj = min_leaf_obj
        self.max_tree_leafs = max_tree_leafs

    def get_depth(self, node) :
        if node is None :
            return 0
        return 1 + max(self.get_depth(node.left), self.get_depth(node.right))

    def get_leaf_count(self, node) :
      if node is None :
          return 0
      if node.left is None and node.right is None :
          return 1
      return self.get_leaf_count(node.left) + self.get_leaf_count(node.right)

    def check_depth_stop(self, node):
        stop = self.get_depth(self.tree) > self.max_tree_depth
        return stop

    def check_min_leaf_stop(self, x1, x2):
        stop = (len(x1) < self.min_leaf_obj or len(x2) < self.min_leaf_obj)
        return stop

    def check_max_leaf_stop(self, node):
        stop = self.get_leaf_count(node) > self.max_tree_leafs
        return stop


    def _chance(self, Y):
        chances = [] 
        for Y_type in set(y_train):
          chances.append(len(Y[Y == Y_type]) / len(Y))
        return chances

    def _ig(self, Rv, Rleft, Rright): 
        return entropy(Rv) - ((len(Rleft) / len(Rv) * entropy(Rleft)) + (len(Rright) / len(Rv) * entropy(Rright)))

    def _split(self, X, Y, node, criter):
        if len(set(Y)) == 1:
            node.bad = True
            return

        best_split_j, best_t, best_ig = -1, -1, -1
        best_lp, best_rp, x1, x2 = [], [], [], []

        for j, x in enumerate(X.T): 
            for t in x: 
                left_part, right_part = Y[x < t], Y[x >= t] 
                if len(left_part) == 0 or len(right_part) == 0: 
                    continue
                ig = self._ig(Y, left_part, right_part) 
                if ig > best_ig: 
                    best_t, best_ig, best_lp, best_rp, best_split_j = t, ig, left_part, right_part, j
                    x1 = X[x >= best_t] 
                    x2 = X[x < best_t] 
            if self.check_min_leaf_stop(x1, x2):
                node.bad = True
                return
            node.left = node._grow_node(self._chance(best_lp))
            node.right = node._grow_node(self._chance(best_rp))
            
            if criter == 'max_depth':
                if self.check_depth_stop(node):
                  node.left = node.right = None
            if criter == 'min_samples_leaf':
                if self.check_min_leaf_stop(x1, x2):
                  node.left = node.right = None
            if criter == 'max_leaf_nodes':
                if self.check_max_leaf_stop(node):
                  node.left = node.right = None

            if node.left is None and node.right is None :
                node.bad = True
                return   
            node.j, node.t =  best_split_j, best_t
            self._split(x1, best_rp, node.right, criter) 
            self._split(x2, best_lp, node.left, criter) 

    def fit(self, X, Y, criter):
        self.Y = Y
        self.tree = Node(self._chance(Y))
        self._split(np.asarray(X), Y, self.tree, criter)

    def predict(self, X):
        predicted = []
        for x in X:
          predicted.append(np.argmax(self._predict_node(x, self.tree)))
        return predicted

    def _predict_node(self, X, node):
        if node.bad:
            return node.chances
        elif X[node.j] > node.t:
            return self._predict_node(X, node.right)
        else:
            return self._predict_node(X, node.left)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=230)

In [ ]:
tree1 = Tree()
tree1.fit(X_train, y_train, 'min_samples_leaf')
y_pred_min_samples_leaf = tree1.predict(X_test)
accuracy_min_samples_leaf = accuracy_score(y_test, y_pred_min_samples_leaf)
print('Точность min_samples_leaf =', accuracy_min_samples_leaf)

Точность min_samples_leaf = 0.9


In [ ]:
tree = Tree()
tree.fit(X_train, y_train, 'max_leaf_nodes')
y_pred_max_leaf_nodes = tree.predict(X_test)
accuracy_max_leaf_nodes = accuracy_score(y_test, y_pred_max_leaf_nodes)
print('Точность max_leaf_nodes =', accuracy_max_leaf_nodes)

Точность max_leaf_nodes = 0.9


In [ ]:
tree = Tree()
tree.fit(X_train, y_train, 'max_depth')
y_pred_max_depth = tree.predict(X_test)
accuracy_max_depth = accuracy_score(y_test, y_pred_max_depth)
print('Точность max_depth =', accuracy_max_depth)

Точность max_depth = 0.9


Sklearn(без спецификации критерия остановки):

In [ ]:
from sklearn.tree import DecisionTreeClassifier

simple_tree = DecisionTreeClassifier()

In [ ]:
simple_tree.fit(X_train, y_train)
simple_tree.score(X_test, y_test)

0.9333333333333333

Sklearn (для разных критериев остановки):

In [ ]:
from sklearn.model_selection import GridSearchCV

tree_params = {'max_depth': np.arange(1, 11)}
tree_grid = GridSearchCV(simple_tree, tree_params)
tree_grid.fit(X_train, y_train)
print('Параметр:', tree_grid.best_params_)
print('Точность:', tree_grid.score(X_test, y_test))
print('Точность моей модели:', accuracy_max_depth)

Параметр: {'max_depth': 3}
Точность: 0.9333333333333333
Точность моей модели: 0.9


In [ ]:
tree_params = {'min_samples_leaf':np.arange(1, 20)}
tree_grid = GridSearchCV(simple_tree, tree_params)
tree_grid.fit(X_train, y_train)
print('Параметр:', tree_grid.best_params_)
print('Точность:', tree_grid.score(X_test, y_test))
print('Точность моей модели:', accuracy_min_samples_leaf)

Параметр: {'min_samples_leaf': 1}
Точность: 0.9333333333333333
Точность моей модели: 0.9


In [ ]:
tree_params = {'max_leaf_nodes':np.arange(2, 10)}
tree_grid = GridSearchCV(simple_tree, tree_params)
tree_grid.fit(X_train, y_train)
print('Параметр:', tree_grid.best_params_)
print('Точность:', tree_grid.score(X_test, y_test))
print('Точность моей модели:', accuracy_max_leaf_nodes)

Параметр: {'max_leaf_nodes': 4}
Точность: 0.9333333333333333
Точность моей модели: 0.9


Точность моей модели блика к значениям sklearn

##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator
from collections import Counter

In [ ]:
class RandomForest(BaseEstimator):
    def __init__(self, n_trees = 5, criterion = 'entropy', max_depth = 5, min_leaf_size = 5, max_leaf_count = 5):
      self.n_trees = n_trees
      self.trees = []
      self.max_depth = max_depth
      self.min_leaf_size = min_leaf_size
      self.max_leaf_count = max_leaf_count
      self.criterion = criterion

    @staticmethod
    def bootstrap(X, y):
      m = len(X)
      indexes = np.random.choice(m, m, replace = True)
      return X.iloc[indexes, :], y.iloc[indexes]

    def fit(self, X, y):
      if len(self.trees) > 0:
        self.trees = []
      for i in range(self.n_trees) :
          X_with_repl, y_with_repl = self.bootstrap(X, y)
          tree = DecisionTreeClassifier(criterion = self.criterion, max_depth = self.max_depth,
                                        min_samples_leaf=self.min_leaf_size,
                                        max_leaf_nodes = self.max_leaf_count,
                                        max_features = 'sqrt')
          self.trees.append(tree.fit(X_with_repl, y_with_repl))
      return self

    def predict(self, X) :
      Y = np.array([tree.predict(X)  for tree in self.trees])

      y_predicted = np.array([])
      for prediction in Y.T:
        counter = Counter(prediction)
        y_predicted = np.append(y_predicted, counter.most_common(1)[0][0])
      return y_predicted

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ML/churn.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
!pip install --upgrade category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 659 kB/s 


Предобработка данных

In [ ]:
import category_encoders
from category_encoders import TargetEncoder

data = data.drop(['RowNumber','CustomerId','Surname'], axis = 1)
encoder = TargetEncoder()
param_to_encode = ['Geography', 'Gender']

data[param_to_encode] = encoder.fit_transform(data[param_to_encode], data['Exited'])
data.head(5)

/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:124: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:129: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0.161548,0.250715,42,2,0.00,1,1,1,101348.88,1
1,608,0.166734,0.250715,41,1,83807.86,1,0,1,112542.58,0
2,502,0.161548,0.250715,42,8,159660.80,3,1,0,113931.57,1
3,699,0.161548,0.250715,39,1,0.00,2,0,0,93826.63,0
4,850,0.166734,0.250715,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
X = data.drop('Exited', axis = 1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=239)

forest = RandomForest()
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print('accuracy:', accuracy)

accuracy: 0.797


In [ ]:
RandomForest().get_params().keys()

dict_keys(['criterion', 'max_depth', 'max_leaf_count', 'min_leaf_size', 'n_trees'])

Найдем наилучшие гиперпараметры

In [ ]:
from sklearn.metrics import accuracy_score

forest_2 = RandomForest()
forest_params = {'n_trees': np.arange(5, 50, 5),
                'max_depth': np.arange(2, 21, 4), 
                'min_leaf_size': np.arange(1, 20, 5), 
                'criterion': ['entropy', 'gini'],
                'max_leaf_count': np.arange(2, 21, 4)}

In [ ]:
forest_grid = GridSearchCV(forest, forest_params, scoring = 'accuracy')

In [ ]:
%time
forest_grid.fit(X_train, y_train)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.54 µs


GridSearchCV(estimator=RandomForest(),
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': array([ 2,  6, 10, 14, 18]),
                         'max_leaf_count': array([ 2,  6, 10, 14, 18]),
                         'min_leaf_size': array([ 1,  6, 11, 16]),
                         'n_trees': array([ 5, 10, 15, 20, 25, 30, 35, 40, 45])},
             scoring='accuracy')

In [ ]:
print('Точность модели:', forest_grid.score(X_test, y_test))
print('\nПодобранные параметры:', forest_grid.best_params_)
print('\nВажность параметров:', forest_grid.best_estimator_.trees[0].feature_importances_)

Точность модели: 0.852

Подобранные параметры: {'criterion': 'gini', 'max_depth': 14, 'max_leaf_count': 18, 'min_leaf_size': 16, 'n_trees': 30}

Важность параметров: [0.00439021 0.10775539 0.         0.39322865 0.         0.04790759
 0.4392978  0.         0.         0.00742036]


In [ ]:
best_model_trees = forest_grid.best_estimator_.trees
n_trees = len(best_model_trees)

sum = 0
for i in range(n_trees):
  sum += best_model_trees[i].feature_importances_
importances = sum/n_trees
importances

array([0.01010196, 0.09081393, 0.01124883, 0.39883116, 0.00145627,
       0.05379399, 0.31728011, 0.00056576, 0.10918124, 0.00672675])

In [ ]:
feature_importances = pd.DataFrame(index = X.columns, data = {'Важность признака' : importances}).sort_values(by = 'Важность признака', ascending = False)
print(feature_importances)

                 Важность признака
Age                       0.398831
NumOfProducts             0.317280
IsActiveMember            0.109181
Geography                 0.090814
Balance                   0.053794
Gender                    0.011249
CreditScore               0.010102
EstimatedSalary           0.006727
Tenure                    0.001456
HasCrCard                 0.000566


Значения признаков "Age", "NumOfProducts", "IsActiveMember", "Geography", "Balance" - являются самыми важными индикаторами того, что банк потеряет клиента.